## Decision Trees

In [1]:
#importing libraries
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps
eps

2.220446049250313e-16

## Data

In [3]:
data = {'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain' ],
        'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
        'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
        'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak',  'Strong', 'Strong', 'Weak', 'Strong'],
        'Play': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No',  'Yes', 'Yes',  'Yes', 'Yes', 'Yes', 'No' ]}
df = pd.DataFrame(data)

In [4]:
df.shape

(14, 5)

In [5]:
df.values

array([['Sunny', 'Hot', 'High', 'Weak', 'No'],
       ['Sunny', 'Hot', 'High', 'Strong', 'No'],
       ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
       ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
       ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
       ['Sunny', 'Mild', 'High', 'Weak', 'No'],
       ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
       ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
       ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
       ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
       ['Rain', 'Mild', 'High', 'Strong', 'No']], dtype=object)

In [6]:
df.keys()

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play'], dtype='object')

In [7]:
target = df.keys()[-1]
target

'Play'

In [8]:
df[target].unique()

array(['No', 'Yes'], dtype=object)

In [9]:
df[target].value_counts()

Yes    9
No     5
Name: Play, dtype: int64

## Implementation

### Calculating entropy of the whole dataset

In [10]:
def find_entropy_whole(df):
    #last column in the dataframe is the target variable
    target = df.keys()[-1]

    #initialisation
    overall_entropy = 0

    #possible values of target
    values_in_target = df[target].unique()

    for value in values_in_target:
        p = df[target].value_counts()[value] / len(df[target])
        overall_entropy += -p*np.log2(p)
    
    return overall_entropy

find_entropy_whole(df)

0.9402859586706309

### Calculating entropy of an attribute

In [11]:
def find_entropy_of_attribute(df, attribute):
    target = df.keys()[-1]
    values_in_target = df[target].unique()
    values_in_attribute = df[attribute].unique()

    entropy_attribute = 0

    for attribute_value in values_in_attribute:
        overall_entropy = 0

        for target_value in values_in_target:
            num = len(df[attribute][df[attribute] == attribute_value][df[target] == target_value])
            den = len(df[attribute][df[attribute] == attribute_value])

            p = num/(den + eps)
            overall_entropy += -p * np.log2(p + eps) 
        p2 = den / len(df)
        entropy_attribute += -p2 * overall_entropy

    return abs(entropy_attribute) 

In [12]:
for i in df.keys()[:-1]:
    print(f'Entropy of attribute \'{i}\' is :', find_entropy_of_attribute(df, i))

Entropy of attribute 'Outlook' is : 0.6935361388961914
Entropy of attribute 'Temperature' is : 0.9110633930116756
Entropy of attribute 'Humidity' is : 0.7884504573082889
Entropy of attribute 'Wind' is : 0.892158928262361


In [14]:
#Finding the best attribute
def find_best_attribute_to_divide(df):
    #Information gain
    IG = []

    all_attribute_names = df.keys()[:-1]

    for attr in all_attribute_names:
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attr))
    
    index_of_attr_with_max_IG = np.argmax(IG)

    best_attribute = all_attribute_names[index_of_attr_with_max_IG]

    return best_attribute

In [15]:
find_best_attribute_to_divide(df)

'Outlook'

## Building decision tree

In [16]:
def buildTree(df, tree=None):

    target = df.keys()[-1]

    #Get attribute with mximum information gain
    node = find_best_attribute_to_divide(df)

    # get distinct value of that attribute
    attValue = np.unique(df[node])

    # Create an empty dictionary to create tree
    if tree is None:
        tree = {}
        tree[node] = {}

    # We make loop to construct a tree by calling this function recursively
    for value in attValue:

        subtable = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtable[target], return_counts=True)

        if len(counts) == 1: #Checking purity of subset
            tree[node][value] = clValue[0]
        else:
            tree[node][value] = buildTree(subtable) #recursive function call

    return tree

buildTree(df)

{'Outlook': {'Overcast': 'Yes',
  'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
  'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}

ID3 in its pure form performs no backtracking in its search. Once it selects an attribute to test at a particular level in the tree, it ever back tracks to reconsider this choice. Therefore it is susceptible to the usual risks of hill-climbing search without backtracking: converging to a locally optimal solutions that are not globally optimal.